In [1]:
import pprint
import time

import pandas
import numpy

import plotly.express as pltxpr

import cmapPy.pandasGEXpress.GCToo as GCToo
import cmapPy.pandasGEXpress.write_gct as write_gct

In [4]:
input_armies = "possible_armies_IPC_30_r66x6.txt"
input_metrics = "output/data/30_IPC_army/all_result_metrics_r4356000x3.txt.gz"

In [29]:
output_stats_template = "output/data/30_IPC_army/stats_result_metrics_r{}x{}.txt"
output_heatmap_template = "output/data/30_IPC_army/heatmap_{}_result_r{}x{}.gct"

In [25]:
armies_df = pandas.read_csv(input_armies, sep="\t")
armies_df.index.name = "army_id"
armies_df

,inf,art,tank,fight,bomb,cost
army_id,,,,,,
0,0,0,0,3,0,30
1,0,0,1,0,2,30
2,0,0,1,1,1,28
3,0,0,3,0,1,30
4,0,0,3,1,0,28
...,...,...,...,...,...,...
61,6,3,0,0,0,30
62,7,2,0,0,0,29
63,8,0,1,0,0,30


In [3]:
result_df = pandas.read_csv(input_metrics, sep="\t")
result_df

,attack_army_id,defense_army_id,fraction_ipc_winner
0,0,0,-0.333333
1,0,0,0.333333
2,0,0,0.333333
3,0,0,0.666667
4,0,0,0.666667
...,...,...,...
4355995,65,65,-0.600000
4355996,65,65,-0.200000
4355997,65,65,-0.600000
4355998,65,65,-0.900000


In [11]:
cols = ["attack_army_id", "defense_army_id"]

g = result_df[cols+["fraction_ipc_winner"]].groupby(cols)
t = g.median()
t.columns = ["median_fraction_ipc_winner"]

u = g.min()
u.columns = ["min_fraction_ipc_winner"]

k = g.max()
k.columns = ["max_fraction_ipc_winner"]

v = g.quantile([0.25, 0.75])
# display(v)
w = v.reset_index().pivot(index=["attack_army_id","defense_army_id"], columns="level_2")
w.columns = [str(x[1]).replace("0.", "percentile_") for x in w.columns]
w["iqr_fraction_ipc_winner"] = w.percentile_75 - w.percentile_25
# w

stat_result_df = pandas.concat([t,u,k,w], axis=1)

stat_result_df["mod_t_fraction_ipc_winner"] = stat_result_df["median_fraction_ipc_winner"] / stat_result_df["iqr_fraction_ipc_winner"]
stat_result_df["abs_mod_t_fraction_ipc_winner"] = stat_result_df.mod_t_fraction_ipc_winner.abs()

stat_result_df.sort_values(
    ["median_fraction_ipc_winner", "iqr_fraction_ipc_winner", "min_fraction_ipc_winner"], inplace=True,
    ascending=[True, False, True]
)
stat_result_df

median_fraction_ipc_winner  \
attack_army_id defense_army_id                               
0              30                                     -0.8   
               32                                     -0.8   
2              32                                     -0.8   
6              32                                     -0.8   
29             32                                     -0.8   
...                                                    ...   
36             6                                       0.9   
41             6                                       0.9   
53             6                                       0.9   
61             6                                       0.9   
49             6                                       0.9   

                                min_fraction_ipc_winner  \
attack_army_id defense_army_id                            
0              30                             -1.000000   
               32                             -1.000000   
2              32                             -1.000000   
6              32                             -1.000000   
29             32                             -1.000000   
...                                                 ...   
36             6                               0.533333   
41             6                               0.533333   
53             6                               0.600000   
61             6                               0.600000   
49             6                               0.700000   

                                max_fraction_ipc_winner  percentile_25  \
attack_army_id defense_army_id                                           
0              30                              1.000000           -0.9   
               32                              0.333333           -0.9   
2              32                              0.785714           -0.9   
6              32                              0.857143           -0.9   
29             32                              0.892857           -0.9   
...                                                 ...            ...   
36             6                               1.000000            0.9   
41             6                               1.000000            0.9   
53             6                               1.000000            0.9   
61             6                               1.000000            0.9   
49             6                               1.000000            0.9   

                                percentile_75  iqr_fraction_ipc_winner  \
attack_army_id defense_army_id                                           
0              30                        -0.6                      0.3   
               32                        -0.6                      0.3   
2              32                        -0.6                      0.3   
6              32                        -0.6                      0.3   
29             32                        -0.6                      0.3   
...                                       ...                      ...   
36             6                          1.0                      0.1   
41             6                          1.0                      0.1   
53             6                          1.0                      0.1   
61             6                          1.0                      0.1   
49             6                          1.0                      0.1   

                                mod_t_fraction_ipc_winner  \
attack_army_id defense_army_id                              
0              30                               -2.666667   
               32                               -2.666667   
2              32                               -2.666667   
6              32                               -2.666667   
29             32                               -2.666667   
...                                                   ...   
36             6                           

In [15]:
output_file = output_stats_template.format(stat_result_df.shape[0], stat_result_df.shape[1])
print(output_file)
stat_result_df.to_csv(output_file, sep="\t")

output/data/30_IPC_army/stats_result_metrics_r4356x8.txt


In [16]:
stat_result_df.sort_values("abs_mod_t_fraction_ipc_winner")

,,median_fraction_ipc_winner,min_fraction_ipc_winner,max_fraction_ipc_winner,percentile_25,percentile_75,iqr_fraction_ipc_winner,mod_t_fraction_ipc_winner,abs_mod_t_fraction_ipc_winner
attack_army_id,defense_army_id,,,,,,,,
55,38,0.0,-1.0,1.000000,-0.642857,0.586207,1.229064,0.0,0.0
20,15,0.0,-1.0,1.000000,-0.600000,0.428571,1.028571,0.0,0.0
54,49,0.0,-1.0,1.000000,-0.600000,0.428571,1.028571,0.0,0.0
44,18,0.0,-1.0,0.896552,-0.466667,0.551724,1.018391,0.0,0.0
36,26,0.0,-1.0,1.000000,-0.482759,0.533333,1.016092,0.0,0.0
...,...,...,...,...,...,...,...,...,...
32,28,0.9,0.0,1.000000,0.800000,0.900000,0.100000,9.0,9.0
59,28,0.9,0.0,1.000000,0.800000,0.900000,0.100000,9.0,9.0
63,28,0.9,0.0,1.000000,0.800000,0.900000,0.100000,9.0,9.0


In [17]:
f = numpy.linspace(0., 1., num=stat_result_df.shape[0])

for col in stat_result_df.columns[:]:
    title = "{} ECDF 30 IPC army matchups".format(col)
    print(title)
    t = stat_result_df[col].sort_values()
    hover = ["a{} d{}".format(x[0], x[1]) for x in t.index]
    fig = pltxpr.scatter(
        x=t, y=f, hover_data=[hover], title=title+"<br>N={}".format(stat_result_df.shape[0]),
        labels={"x":col, "y":"fraction of matchups"}
    )
    fig.write_html("output/figures/30_IPC_army/ECDF/{}.html".format(title), include_plotlyjs="cdn")
    

median_fraction_ipc_winner ECDF 30 IPC army matchups
min_fraction_ipc_winner ECDF 30 IPC army matchups
max_fraction_ipc_winner ECDF 30 IPC army matchups
percentile_25 ECDF 30 IPC army matchups
percentile_75 ECDF 30 IPC army matchups
iqr_fraction_ipc_winner ECDF 30 IPC army matchups
mod_t_fraction_ipc_winner ECDF 30 IPC army matchups
abs_mod_t_fraction_ipc_winner ECDF 30 IPC army matchups


In [18]:
t = stat_result_df.sort_values(["median_fraction_ipc_winner", "mod_t_fraction_ipc_winner"])

title = "median and iqr results for 30 IPC army matchups"
fig = pltxpr.scatter(
    t, x=range(t.shape[0]), y="median_fraction_ipc_winner", error_y=t.percentile_75-t.median_fraction_ipc_winner,
    error_y_minus=t.median_fraction_ipc_winner-t.percentile_25,
    title=title
)
fig.write_html("output/figures/30_IPC_army/{}.html".format(title), include_plotlyjs="cdn")

In [19]:
t = stat_result_df.sort_values(["median_fraction_ipc_winner", "mod_t_fraction_ipc_winner"])

title = "median and iqr results for 30 IPC army matchups"
fig = pltxpr.scatter(
    t, x=range(t.shape[0]), y="median_fraction_ipc_winner", error_y=t.percentile_75-t.median_fraction_ipc_winner,
    error_y_minus=t.median_fraction_ipc_winner-t.percentile_25,
    title=title
)
fig.write_html("output/figures/30_IPC_army/{}.html".format(title), include_plotlyjs="cdn")

In [20]:
t = stat_result_df[["median_fraction_ipc_winner"]].reset_index().pivot(
    index="attack_army_id", columns="defense_army_id")
t.columns = [x[1] for x in t.columns]
t.columns.name = "defense_army_id"
heatmap_median_result_df = t
heatmap_median_result_df

defense_army_id,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
attack_army_id,,,,,,,,,,,,,,,,,,,,,
0,-0.333333,0.666667,0.333333,-0.500000,-0.571429,-0.600000,0.666667,-0.666667,-0.428571,-0.642857,...,-0.793103,-0.800000,-0.785714,-0.8,-0.785714,-0.800000,-0.793103,-0.8,-0.785714,-0.8
1,0.000000,0.800000,0.400000,-0.400000,-0.571429,-0.600000,0.800000,-0.666667,0.400000,-0.642857,...,-0.793103,-0.750000,-0.785714,-0.8,-0.785714,-0.800000,-0.793103,-0.8,-0.785714,-0.8
2,-0.333333,0.785714,0.428571,-0.400000,-0.571429,-0.600000,0.785714,-0.666667,0.000000,-0.642857,...,-0.793103,-0.800000,-0.785714,-0.8,-0.785714,-0.800000,-0.793103,-0.8,-0.785714,-0.8
3,0.400000,0.800000,0.600000,0.600000,0.000000,-0.400000,0.800000,0.000000,0.600000,-0.428571,...,-0.689655,-0.600000,-0.678571,-0.7,-0.678571,-0.700000,-0.689655,-0.7,-0.678571,-0.7
4,0.357143,0.785714,0.571429,0.571429,-0.357143,-0.400000,0.785714,-0.333333,0.571429,-0.428571,...,-0.689655,-0.700000,-0.678571,-0.7,-0.678571,-0.700000,-0.689655,-0.8,-0.678571,-0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.700000,0.900000,0.800000,0.800000,0.700000,0.600000,0.900000,0.700000,0.800000,0.600000,...,0.133333,0.600000,0.266667,-0.2,0.133333,-0.266667,-0.275862,-0.4,-0.250000,-0.4
62,0.689655,0.896552,0.793103,0.793103,0.586207,0.482759,0.896552,0.586207,0.793103,0.586207,...,-0.206897,0.482759,0.275862,-0.4,-0.214286,-0.400000,-0.379310,-0.5,-0.357143,-0.5
63,0.700000,0.900000,0.800000,0.700000,0.600000,0.500000,0.900000,0.600000,0.800000,0.500000,...,-0.344828,0.400000,-0.357143,-0.5,-0.464286,-0.500000,-0.482759,-0.5,-0.464286,-0.6


In [21]:
sum_defense_median_fraction_ipc_winner = heatmap_median_result_df.sum(axis=0)
sum_defense_median_fraction_ipc_winner.name = "sum_defense_median_fraction_ipc_winner"
sum_defense_median_fraction_ipc_winner

defense_army_id
0     37.109031
1     57.917323
2     48.793350
3     41.227750
4     23.320197
        ...    
61   -40.733333
62   -39.931034
63   -42.050000
64   -38.732143
65   -43.300000
Name: sum_defense_median_fraction_ipc_winner, Length: 66, dtype: float64

In [22]:
sum_attack_median_fraction_ipc_winner = heatmap_median_result_df.sum(axis=1)
sum_attack_median_fraction_ipc_winner.name = "sum_attack_median_fraction_ipc_winner"
sum_attack_median_fraction_ipc_winner

attack_army_id
0    -43.136617
1    -39.466667
2    -41.221429
3    -20.872250
4    -25.047455
        ...    
61    31.774138
62    25.083744
63    15.742447
64    13.486905
65     7.706158
Name: sum_attack_median_fraction_ipc_winner, Length: 66, dtype: float64

In [23]:
combined_attack_defense_median = sum_attack_median_fraction_ipc_winner - sum_defense_median_fraction_ipc_winner
combined_attack_defense_median.name = "combined_attack_defense_median"
combined_attack_defense_median.index.name = "army_id"
display(combined_attack_defense_median)

army_id
0    -80.245649
1    -97.383990
2    -90.014778
3    -62.100000
4    -48.367652
        ...    
61    72.507471
62    65.014778
63    57.792447
64    52.219048
65    51.006158
Name: combined_attack_defense_median, Length: 66, dtype: float64

In [26]:
defense_meta_df = pandas.concat(
    [armies_df, sum_defense_median_fraction_ipc_winner, combined_attack_defense_median], axis=1
)
defense_meta_df

,inf,art,tank,fight,bomb,cost,sum_defense_median_fraction_ipc_winner,combined_attack_defense_median
0,0,0,0,3,0,30,37.109031,-80.245649
1,0,0,1,0,2,30,57.917323,-97.383990
2,0,0,1,1,1,28,48.793350,-90.014778
3,0,0,3,0,1,30,41.227750,-62.100000
4,0,0,3,1,0,28,23.320197,-48.367652
...,...,...,...,...,...,...,...,...
61,6,3,0,0,0,30,-40.733333,72.507471
62,7,2,0,0,0,29,-39.931034,65.014778
63,8,0,1,0,0,30,-42.050000,57.792447
64,8,1,0,0,0,28,-38.732143,52.219048


In [27]:
attack_meta_df = pandas.concat(
    [armies_df, sum_attack_median_fraction_ipc_winner, combined_attack_defense_median], axis=1
)
attack_meta_df

,inf,art,tank,fight,bomb,cost,sum_attack_median_fraction_ipc_winner,combined_attack_defense_median
army_id,,,,,,,,
0,0,0,0,3,0,30,-43.136617,-80.245649
1,0,0,1,0,2,30,-39.466667,-97.383990
2,0,0,1,1,1,28,-41.221429,-90.014778
3,0,0,3,0,1,30,-20.872250,-62.100000
4,0,0,3,1,0,28,-25.047455,-48.367652
...,...,...,...,...,...,...,...,...
61,6,3,0,0,0,30,31.774138,72.507471
62,7,2,0,0,0,29,25.083744,65.014778
63,8,0,1,0,0,30,15.742447,57.792447


In [28]:
heatmap_g = GCToo.GCToo(heatmap_median_result_df, col_metadata_df=defense_meta_df, row_metadata_df=attack_meta_df)
print(heatmap_g)

None
src: None
data_df: [66 rows x 66 columns]
row_metadata_df: [66 rows x 8 columns]
col_metadata_df: [66 rows x 8 columns]


In [31]:
output_file = output_heatmap_template.format("median", heatmap_median_result_df.shape[0], heatmap_median_result_df.shape[1])
print(output_file)
write_gct.write(heatmap_g, output_file)

output/data/30_IPC_army/heatmap_median_result_r66x66.gct
